In [1]:
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm

## 전략

In [2]:
def MFIprocess(data,term):
    
    if len(data) < 120: # 데이터 양이 적다면 에러처리
        return 'error'
    temp2 = data.copy()
    #temp2 = data.iloc[:-360,:]
    temp2 = temp2.iloc[:-(term+1),:] # 인덱스 초과오류방지 -> 최근 데이터는 빼기.
    
    #매수기준 1: 전날의 매수신호
    temp2 = temp2[temp2['MFI'] <= 0.2].dropna() #0.2 미만일 경우, 과대 매도로 판단 저평가 매수
    if len(temp2) == 0:
        return 'error'
    
    #매수기준 2: 그날 2.5% 이상 상승한다면
    temp = data.loc[temp2.index+1,:] 
    temp = temp[temp['일수익률'] >=  0.5].dropna()
    

    buy_point = [-term]
    for i in temp.index:
        if i >= buy_point[-1] + term : # 매수시점이 매도시점과 겹치지 않게
            buy_point.append(i)
    buy_point = buy_point[1:]
    
    #수익률 계산
    res = []
    for i in buy_point: 
        profit = data.loc[i+term,'close price'] - data.loc[i,'close price']
        percentage = round(profit / data.loc[i,'close price'],2)
        res.append(percentage)
   
    if len(res) == 0: # 맞는 조건이 하나도 없다면 에러처리
        return 'error'
    return res

In [107]:
def r_sigma(data,term):
    
    if len(data) < 120: # 데이터 양이 적다면 에러처리
        return 'error'
    temp2 = data.copy()
    #temp2 = data.iloc[:-360,:]
    temp2 = temp2.iloc[:-(term+2),:] # 인덱스 초과오류방지 -> 최근 데이터는 빼기.
    
    #매수기준 1: 전날의 매수신호
    temp2 = temp2[(temp2['williams'] < -80) & (temp2['sigma'] < -2) & (temp2['ma20'] < temp2['ma60'])].dropna() 
    if len(temp2) == 0:
        return 'error'
    
    #매수기준 2: 그날 2.5% 이상 상승한다면
    temp = data.loc[temp2.index+1,:] 
    temp = temp[temp['일수익률'] >=  2.5].dropna()  #매수기준 2: 그날 3% 이상 상승한다면
    
    buy_point = [-term]
    for i in temp.index:
        if i >= buy_point[-1] + term : # 매수시점이 매도시점과 겹치지 않게
            buy_point.append(i)
    buy_point = buy_point[1:]
    
    #수익률 계산
    res = []
    for i in buy_point: 
        profit = data.loc[i+term,'close price'] - data.loc[i,'close price']
        percentage = round(profit / data.loc[i,'close price'],2)
        res.append(percentage)
   
    if len(res) == 0: # 맞는 조건이 하나도 없다면 에러처리
        return 'error'
    return res

In [108]:
def gold_cross(data,term):
    
    if len(data) < 120: # 데이터 양이 적다면 에러처리
        return 'error'
    temp2 = data.copy()
    #temp2 = data.iloc[:-360,:]
    temp2 = temp2.iloc[:-(term+1),:] # 인덱스 초과오류방지 -> 최근 데이터는 빼기.
    
    temp2 = temp2[(temp2['ma5'] < temp2['ma20']) & (temp2['ma5'] > temp2['ma60']) & (temp2['ma5'] > temp2['ma120'])].dropna()  
    
    temp = data.loc[temp2.index+1,:] 
    temp = temp[(temp['ma5'] >= temp['ma20']) & (temp['일수익률'] >= 2)].dropna()   
    
    buy_point = [-term]
    for i in temp.index:
        if i >= buy_point[-1] + term : # 매수시점이 매도시점과 겹치지 않게
            buy_point.append(i)
    buy_point = buy_point[1:]
    
    #수익률 계산
    res = []
    for i in buy_point: 
        profit = data.loc[i+term,'close price'] - data.loc[i,'close price']
        percentage = round(profit / data.loc[i,'close price'],2)
        res.append(percentage)
   
    if len(res) == 0: # 맞는 조건이 하나도 없다면 에러처리
        return 'error'
    return res

In [12]:
def attention(data,term):
    
    if len(data) < 120: # 데이터 양이 적다면 에러처리
        return 'error'
    
    temp2 = data.copy()
    #temp2 = data.iloc[:-360,:]
    temp2 = temp2.iloc[:-(term+1),:] # 인덱스 초과오류방지 -> 최근 데이터는 빼기.
    temp2 = temp2[(temp2['amount'] >= temp2['amount_ma5'] * 3) & (temp2['일수익률'] >= 1)].dropna()  
    
    
    buy_point = [-term]
    for i in temp2.index:
        if i >= buy_point[-1] + term : # 매수시점이 매도시점과 겹치지 않게
            buy_point.append(i)
    buy_point = buy_point[1:]
    
    #수익률 계산
    res = []
    for i in buy_point: 
        profit = data.loc[i+term,'close price'] - data.loc[i,'close price']
        percentage = round(profit / data.loc[i,'close price'],2)
        res.append(percentage)
   
    if len(res) == 0: # 맞는 조건이 하나도 없다면 에러처리
        return 'error'
    return res

In [10]:
def long_candle(data,term,a =5,b=5):
    if len(data) < 120: # 데이터 양이 적다면 에러처리
        return 'error'
    temp = data.copy()
    #temp = data.iloc[:-360,:]
    temp = temp.iloc[:-(term+1)] # 인덱스 초과 오류방지 (term 기간만큼 삭제)
    
    # 당일 종가가 전일 종가의 a% 이상 상승
    temp['preDayClose'] = temp['close price'].shift(1)
    cond1 = temp['close price'] > temp['preDayClose']*(1+a/100)

    # 장대 양봉 (b% 이상 차이)
    cond2 = temp['high price'] > temp['low price'] * (1+b/100)

    # 거래량 급증 (5일 평균보다 높을 때)
    temp['amountMa5'] = temp['amount'].rolling(window=5).mean()
    cond3 = temp['amount'] >= temp['amountMa5'] * 2.5
    
    # # 꽉찬 장대
    cond4 = temp['high price'] == temp['close price']
    # cond5 = temp['low price'] == temp['start price']

    buy_day = temp[cond1&cond2&cond3]
    buy_day = data.loc[buy_day.index+1,:]

    buy_point = [-term]
    for i in buy_day.index:
        if i >= buy_point[-1] + term : # 매수시점이 매도시점과 겹치지 않게
            buy_point.append(i)
    buy_point = buy_point[1:]

    #수익률 계산
    res = []
    for i in buy_point:
        profit = data.loc[i+term,'close price'] - data.loc[i,'close price']
        percentage = round(profit / data.loc[i,'close price'],2)
        res.append(percentage)
   
    if len(res) == 0: # 맞는 조건이 하나도 없다면 에러처리
        return 'error'
    return res

In [24]:
def bollinger(data,term):
    
    if len(data) < 120: # 데이터 양이 적다면 에러처리
        return 'error'
    
    temp2 = data.copy()
    #temp2 = data.iloc[:-360,:]
    temp2 = temp2.iloc[:-(term+1),:] # 인덱스 초과오류방지 -> 최근 데이터는 빼기.
    temp2 = temp2[temp2['close price'] <= (temp2['ma20']- 2.1 * temp2['std'])].dropna()
    temp2 = temp2[temp2['high price'] > (temp2['ma20']- 2.1 * temp2['std'])].dropna()
    
    
    
    buy_point = [-term]
    for i in temp2.index:
        if i >= buy_point[-1] + term : # 매수시점이 매도시점과 겹치지 않게
            buy_point.append(i)
    buy_point = buy_point[1:]
    
    #수익률 계산
    res = []
    for i in buy_point: 
        profit = data.loc[i+term,'close price'] - data.loc[i,'close price']
        percentage = round(profit / data.loc[i,'close price'],2)
        res.append(percentage)
   
    if len(res) == 0: # 맞는 조건이 하나도 없다면 에러처리
        return 'error'
    return res

## 테스트

In [25]:
data_list = os.listdir("./data/stock")
total = []
total_cnt = 0
for i in tqdm(data_list):
    win = 0
    data_dir = "./data/stock/" +i
    df1 = pd.read_csv(data_dir,encoding='cp949')
    ans=  bollinger(df1,5)
    if ans != 'error':
        total.append( sum(ans) / len(ans) * 100)
        total_cnt += len(ans)
        print(ans)
print("전체 주식종목 대비 기간수익률 : {}%".format(round(sum(total) / len(total),2)))
print("매수신호 횟수 : {}회".format(total_cnt))

[-0.11, 0.0, 0.1, -0.06, -0.03, 0.01, 0.0, 0.01, -0.01, -0.01, 0.04, -0.23, 0.11, -0.02, -0.0, 0.06]
[0.02, -0.0, -0.08, 0.01, 0.05, 0.06, -0.02, 0.03, 0.13, -0.04, -0.02, 0.03, -0.02, 0.02, 0.02, 0.05, -0.05, -0.03, 0.0, 0.08, -0.27, 0.02, 0.06, -0.01, 0.0, 0.0, -0.02, -0.01, 0.01, 0.04, -0.01, 0.0, 0.01, -0.03, -0.03, -0.02, -0.04, -0.03, 0.02, 0.02, -0.02, -0.16, -0.2, 0.07, 0.01, 0.03, 0.03]
[-0.01, 0.04, 0.02, 0.06, -0.06, 0.09, -0.0, -0.0, 0.03, 0.01, 0.02, 0.15, 0.03, 0.03, -0.01, 0.09, -0.02, 0.08, -0.04, 0.06, 0.0, 0.0, -0.04, 0.05, -0.06, -0.05, 0.04, 0.02, -0.06, 0.03, 0.0, 0.07, -0.03, -0.04, 0.01, -0.02, -0.09, 0.05, -0.01, -0.05, 0.07, -0.02, -0.0, 0.02, 0.01, -0.02, -0.24, 0.02, 0.19, 0.05]
[-0.01, 0.01, 0.01, 0.05, 0.01, 0.02, -0.0, 0.01, 0.0, 0.01, 0.06, -0.01, 0.03, 0.01, 0.06, -0.0, -0.01, 0.01, -0.04, -0.06, -0.03, -0.0, -0.0, 0.03, 0.02, 0.02, 0.09, 0.01, 0.06, 0.01, 0.11, 0.04, -0.01, 0.04, -0.04, 0.04, -0.01, 0.07, -0.02, 0.01, 0.21, -0.01, 0.01]
[0.04, 0.01, 0.0

[-0.01, -0.02, -0.09, -0.04, 0.0, 0.02, 0.04, 0.04, 0.12, -0.02, 0.02, 0.04, -0.01, 0.08, 0.03, -0.02, -0.01, -0.01, 0.03, -0.02, -0.02, 0.02, 0.01, -0.01, 0.07, 0.01, 0.0, -0.04, 0.01, -0.03, -0.05, -0.12, 0.02, -0.02, 0.04, 0.0, -0.03, 0.01, -0.03, -0.02, 0.01, -0.02, -0.1, -0.0, -0.06, 0.02, 0.01, 0.13, 0.03, -0.05, -0.01, 0.0, 0.05, -0.03, -0.33, 0.2, 0.07]
[-0.02, 0.03, -0.02, -0.08, 0.09, -0.04, -0.0, -0.01, 0.01, -0.02, 0.01, -0.01, -0.02, 0.03, -0.03, -0.04, 0.02, -0.01, 0.0, -0.03, -0.12, -0.03, 0.09, 0.03, 0.01, -0.06, -0.02, 0.01, -0.05, -0.01, 0.02, -0.05, 0.02, -0.02, -0.0, -0.04, 0.01, -0.02, 0.03, 0.02, -0.05, 0.03, 0.02, 0.05]
[0.03, -0.0, 0.02, 0.01, 0.0, -0.03, 0.03, 0.11, 0.03, -0.03, 0.02, 0.07, 0.04, 0.01, 0.08, -0.0, 0.02, 0.04, 0.0, -0.02, -0.0, 0.08, -0.07, -0.01, 0.01, 0.01, -0.03, -0.03, -0.05, 0.02, -0.02, -0.01, 0.04, 0.01, -0.01, 0.02, -0.05, -0.06, 0.21, -0.05, 0.02, 0.03, -0.08, -0.08, 0.01]
[0.05, 0.08, 0.02, 0.05, -0.06, 0.02, 0.06, -0.01, 0.08, -0.02, 

[0.03, 0.08, -0.01, 0.12, 0.01, 0.01, 0.02, -0.03, 0.03, -0.03, 0.03, -0.05, 0.0, -0.02, 0.05, -0.01, 0.0, -0.02, 0.03, 0.02, 0.11, 0.1, 0.03, -0.05, -0.05, 0.03, 0.05, 0.06, 0.07, 0.05, 0.02, -0.01, 0.02, -0.05, 0.01, 0.01, 0.01, -0.05, -0.01, 0.04, 0.0, -0.01, -0.17, -0.14, -0.02, 0.01]
[0.02, 0.04, -0.07, 0.05, -0.01, -0.04, -0.16, -0.05, 0.18, -0.01, 0.07, -0.08, -0.24, 0.07, -0.01, -0.25, -0.19, 0.05, -0.09, 0.49, 0.17, 0.11, -0.04, -0.05, 0.01, -0.06, -0.03, -0.04, 0.03, 0.01, 0.05]
[0.07, -0.09, -0.03, 0.01, 0.07, 0.02, 0.03, 0.0, 0.04, -0.05, -0.01, -0.01, 0.15, -0.01, 0.06, -0.02, -0.03, 0.19, -0.05, 0.04, -0.01, 0.03, 0.02, 0.11, 0.02, 0.07, 0.05, 0.1, 0.07, -0.06, 0.01, 0.01, -0.01, -0.02, 0.02, -0.01, 0.01, -0.04, -0.0, -0.1, 0.01, -0.02, -0.02, 0.13, -0.02, 0.08, -0.18]
[0.08, -0.13, 0.06, -0.01, -0.02, -0.03, 0.28, -0.01, 0.0, 0.03, 0.04, 0.02, 0.01, 0.03, 0.09, -0.0, 0.0, -0.04, 0.1, 0.06, 0.07, 0.07, 0.05, 0.11, 0.03, -0.01, -0.03, 0.02, -0.01, 0.09, -0.02, 0.0, 0.02, 0

[0.03, 0.02, 0.04, 0.02, -0.01, 0.05, 0.0, -0.01, -0.03, -0.02, 0.03, 0.01, -0.01, 0.01, 0.04, 0.07, -0.05, 0.1, 0.01, 0.07, -0.01, -0.03, 0.07, 0.05, -0.02, 0.01, 0.05, -0.02, -0.0, -0.04, 0.07, 0.05, 0.01, 0.01, -0.04, 0.03, -0.01, 0.03, -0.06, 0.0, -0.0, 0.01, 0.08, 0.0, -0.01, -0.01, -0.03, 0.08, -0.02]
[-0.03, -0.01, 0.0, -0.01, 0.02, 0.08, -0.02, 0.01, 0.01, 0.13, -0.08, -0.08, 0.15, 0.06, 0.05, 0.03, -0.03, 0.05, -0.01, 0.02, 0.01, -0.12, 0.04, -0.08, 0.01, -0.0, 0.06, 0.01, -0.06, 0.04, -0.0, 0.06, 0.01, -0.01, -0.07, -0.06, 0.02, 0.04, -0.13, -0.09, -0.01, 0.01, -0.25, 0.15, -0.03, -0.05, 0.02, -0.0, 0.06]
[-0.0, -0.02, -0.0, -0.04, 0.17, -0.15, 0.05, 0.02, 0.01, -0.13, 0.09, 0.09, -0.03, -0.01, -0.02, -0.04, -0.07, 0.0, 0.01, 0.07, 0.01, 0.01, 0.01, 0.04, -0.03, 0.1, -0.02, -0.06, -0.11, 0.01, 0.01, -0.04, -0.04, 0.0, -0.01, 0.03, -0.04, 0.13, -0.03, -0.03, -0.1, -0.06, 0.07, 0.11, 0.06, 0.05, 0.04, -0.08, -0.07, -0.04, -0.0, 0.02, 0.03, -0.03, -0.23, 0.13, -0.0, 0.05]
[0.01,

[0.01, 0.01, -0.01, 0.02, 0.02, 0.04, -0.03, -0.01, 0.03, -0.1, -0.06, -0.0, -0.05, -0.05, -0.0, -0.08, 0.08, -0.07, 0.03, 0.06, -0.02, -0.04, 0.03, 0.21, 0.08, 0.08, -0.03, 0.03, -0.11, 0.1, 0.02, 0.01, -0.01, 0.29, 0.08, 0.05, -0.01, 0.01, -0.17, 0.03, 0.01, 0.01, -0.05, 0.0, 0.01, -0.43, 0.16]
[0.03, -0.05, -0.03, 0.06, -0.01, 0.04, 0.01, 0.08, -0.03, -0.04, -0.0, -0.06, -0.0, -0.01, 0.03, 0.01, -0.09, 0.02, -0.07, -0.0, -0.05, -0.03, -0.13, 0.06, -0.03, 0.03, 0.05, 0.02, 0.05, 0.05, 0.07, -0.07, 0.06, -0.03, 0.11, 0.06, 0.0, -0.06, -0.02, 0.02, -0.01, 0.02, -0.14, 0.06]
[-0.01, -0.05, 0.03, -0.05, 0.01, 0.04, -0.01, -0.04, 0.04, -0.06, 0.02, 0.03, 0.01, 0.0, -0.01, 0.0, 0.03, -0.04, 0.02, 0.0, 0.01, 0.08, -0.04, 0.01, -0.03, -0.05, 0.02, -0.17, 0.02]
[-0.03, 0.1, -0.04, 0.09, -0.01, 0.17, 0.01, -0.02, -0.05, -0.03, -0.02, 0.12, 0.02, 0.01, 0.01, -0.06, 0.07]
[0.01, -0.01, -0.06, 0.02, -0.05, -0.0, -0.0, 0.03, -0.0, 0.02, 0.0, 0.07, 0.01, 0.02, -0.0, -0.03, 0.06, -0.02, 0.01, 0.0, 0

[-0.09, 0.04, 0.05, -0.03, 0.04, 0.05, 0.0, -0.1, -0.1, 0.22, 0.02, 0.05, 0.03, -0.03, 0.05, 0.02, 0.06, 0.16, -0.11, -0.06, -0.03, 0.0, 0.06, 0.07, 0.07, 0.05, -0.05, 0.07, -0.03, -0.01, 0.05, -0.1, -0.03, -0.1, -0.13, 0.05, 0.04, -0.04, -0.08, 0.2, 0.08, -0.1, -0.21, -0.12, 0.19, -0.0, 0.03, 0.01, 0.06]
[0.1, 0.05, 0.0, -0.05, 0.0, 0.0, -0.03, 0.08, -0.07, -0.04, -0.0, -0.13, 0.02, 0.09, -0.01, 0.01, 0.03, -0.08, 0.13, 0.06, -0.08, 0.0, 0.05, 0.07, -0.02, -0.01, 0.02, 0.0, -0.04, 0.05, -0.04, -0.01, 0.01, 0.03, 0.08, -0.06, -0.02, -0.12, -0.0, 0.0, -0.01]
[-0.03, -0.04, 0.07, 0.0, 0.01, 0.01, -0.03, 0.01, -0.02, -0.02, 0.01, 0.08, -0.1, 0.12, -0.02, 0.02, -0.06, -0.0, 0.02, 0.04, -0.03, -0.02, 0.08, 0.03, 0.08, -0.03, 0.05, 0.02, -0.07, -0.02, -0.01, 0.02, 0.01, 0.01, -0.09, -0.09, -0.04, -0.18, 0.01, 0.05, 0.07, 0.01, 0.04]
[0.0, -0.07, 0.03, 0.05, 0.13, -0.03, 0.03, 0.01, -0.08, -0.07, -0.02, 0.01, 0.07, 0.01, 0.03, 0.04, -0.03, -0.04, -0.01, -0.05, 0.07, 0.04, -0.01, -0.12, -0.02,

[0.03, 0.1, 0.1, -0.02, 0.14, 0.11, 0.03, -0.01, 0.02, -0.03, 0.26, -0.1, -0.03, 0.02, 0.32, 0.03, 0.02, -0.04, 0.01, 0.04, 0.03, -0.01, 0.05, 0.17, 0.02, -0.07, 0.03, 0.04, -0.04, 0.09, -0.01, 0.02, -0.07, -0.04, 0.49, -0.0, 0.07, 0.05, -0.03, -0.09, 0.04, 0.16, -0.01, 0.09]
[-0.02, -0.0, -0.06, 0.0, 0.07, -0.01, 0.23, -0.02, -0.04, 0.06, 0.02, 0.05, 0.08, -0.02, 0.02, -0.03, 0.08, 0.0, -0.01, 0.02, -0.05, 0.07, -0.1, -0.15, -0.1, 0.11, 0.09, -0.04, 0.01, -0.01, 0.07, -0.02, 0.02, 0.04, -0.06, 0.03, -0.05, -0.06, -0.06, 0.03, -0.06, 0.09, -0.23, 0.2, 0.04]
[0.15, 0.04, 0.08, -0.09, -0.01, -0.01, -0.03, 0.02, 0.03, -0.3, 0.02, 0.08, 0.01, 0.04, -0.11, 0.08, -0.02, -0.02, -0.06, -0.09, -0.01, 0.03, 0.21, 0.02, -0.02, -0.02, 0.06, -0.04, 0.02, 0.08, 0.07, -0.03, -0.02]
[0.02, -0.09, -0.07, -0.03, 0.07, 0.1, -0.08, -0.3, 0.34, 0.08, 0.02]
[-0.0, 0.01, 0.08, 0.0, 0.05, 0.02, 0.06, 0.02, 0.01, -0.07, 0.04, -0.06, -0.09, -0.04, -0.13, 0.04, -0.01, -0.28, 0.11, 0.08]
[-0.0, 0.08, -0.06, 0.23,

[0.38, 0.22, -0.04, 0.04, 0.15, 0.14, -0.06, 0.19, 0.0, 0.04, -0.07, -0.03, -0.06, 0.16, -0.06, -0.02, -0.04, 0.07, -0.03, 0.01, 0.02, -0.07, -0.05, 0.05, -0.03, -0.04, -0.02, -0.02, 0.07, -0.03, 0.08, -0.06, 0.06, 0.03, 0.19, 0.12, 0.09, 0.0, -0.12, 0.01, -0.06, -0.16, 0.01, 0.01, 0.09]
[0.0, 0.01, -0.02, 0.0, -0.05, 0.19, 0.01, 0.09, 0.01, 0.09, 0.07, 0.02, 0.06, -0.03, 0.02, -0.0, -0.03, 0.02, 0.03, 0.05, -0.02, 0.02, 0.04, -0.09, 0.06, 0.05, 0.05, -0.02, 0.01, -0.0, -0.0, -0.2, 0.15]
[0.1, -0.09, -0.0, 0.05, -0.03, 0.08, 0.06]
[-0.14, -0.11, 0.22, 0.09, 0.05]
[-0.06, 0.07, -0.06, -0.01, -0.16, 0.02, -0.12, -0.04, 0.03, -0.01, 0.18, -0.03, 0.2, 0.01, 0.08, 0.05, 0.08, 0.03, 0.08, 0.01, 0.24, -0.08, 0.06, 0.03, 0.03, 0.01, 0.22, 0.01, 0.04, 0.09, -0.08, -0.03, -0.08, 0.01, -0.02, 0.0, -0.07, -0.1, 0.09, -0.31, 0.25, 0.07]
[0.02, 0.02, -0.03, 0.17, 0.01, 0.02, 0.05, 0.04, -0.08, 0.05, 0.06, 0.01, 0.06, 0.01, 0.13, -0.03, -0.01, 0.43, -0.09, 0.06, 0.07, -0.01, 0.02, 0.04, -0.01, 0.05, 

[-0.01, -0.1, -0.05, -0.01, 0.01, 0.05, -0.03, -0.05, 0.02, -0.01, -0.02, 0.01, -0.01, 0.07, -0.01, -0.03, 0.01, -0.05, 0.14, -0.01, 0.03, -0.01, 0.04, -0.02, 0.03, -0.05, -0.01, -0.0, -0.12, 0.13, 0.05, -0.04, -0.07, 0.07, -0.01, 0.02, 0.05, -0.02, 0.03, 0.08, -0.01, -0.06, 0.16, -0.07, 0.07, -0.0, 0.02, -0.02, 0.08, -0.01, -0.01, 0.01, -0.23, 0.04, 0.02, 0.02]
[-0.07, 0.14, 0.16, 0.07, -0.05, 0.04, 0.04, -0.0, 0.01, 0.06, 0.02, 0.01, 0.0, 0.07, 0.0, 0.0, 0.04, -0.01, -0.03, -0.0, -0.03, 0.03, 0.03, -0.08, -0.1, 0.02, 0.01, -0.08, -0.03, 0.02, 0.05, -0.0, 0.02, -0.02, 0.11, 0.01, 0.02, -0.03, -0.04, 0.07, -0.05, 0.09, -0.36, 0.27, 0.08, 0.05]
[0.03, 0.01, 0.03, 0.09, -0.09, 0.03, 0.02, 0.09, -0.09, 0.15, 0.02, -0.0, -0.01, -0.03, -0.03, -0.05, -0.05, 0.07, 0.05, 0.02, -0.01, -0.01, 0.02, 0.0, 0.07, -0.01, -0.05, 0.05, 0.06, 0.03, 0.01, 0.03, 0.06, -0.07, 0.14, -0.06, -0.04, 0.01, -0.12, 0.02, 0.08, -0.03, -0.26, 0.22]
[0.17, 0.08, 0.09, -0.39, 0.22, 0.25, -0.03, -0.12, 0.22]
[-0.11, 0

[0.04, 0.05, 0.04, -0.01, -0.05, -0.18, 0.03, 0.03, 0.0, 0.01, -0.06, -0.0, 0.05, 0.01, 0.04, 0.0, 0.05, 0.01, -0.0, -0.02, -0.0, 0.02, -0.04, -0.01, 0.05, 0.02, -0.05, 0.01, -0.04, -0.02, -0.02, 0.02, -0.03, 0.03, 0.1, -0.02, 0.02, -0.01, 0.01, 0.01, 0.0, -0.0, 0.0, 0.05, 0.01, 0.01, -0.05, 0.03, -0.02, 0.01, -0.06, -0.02, -0.03, -0.03, 0.0, 0.05, 0.03, -0.13, 0.1, -0.03, 0.02]
[0.04, -0.13, 0.32, 0.02, -0.01, -0.03, 0.03, 0.0, -0.03, -0.03, -0.08, -0.02, -0.01, 0.07, 0.04, -0.06, 0.01, -0.04, -0.05, 0.03, -0.05, 0.01, 0.05, 0.05, 0.24, -0.01, 0.14, -0.01, 0.13, 0.02, 0.02, 0.04, 0.08, -0.02, 0.01, 0.07, -0.01, -0.04, -0.05, 0.01, -0.03, -0.03, 0.04, -0.04, -0.22, -0.26, 0.03]
[-0.01, -0.09, -0.19]
[-0.02, -0.02, 0.05, -0.01, -0.02, -0.13, 0.0, -0.01, 0.0, -0.02, -0.01, -0.04, -0.04, -0.03, 0.02, -0.04, -0.03, 0.01, -0.0, 0.01, -0.06, -0.08, 0.03, 0.05, -0.06, -0.07, -0.0, -0.04, 0.06, -0.01, 0.1, 0.02, -0.09, -0.13, 0.04, -0.02, 0.21, 0.02, -0.03, 0.05, 0.01, 0.28, 0.02, 0.01, -0.03,

In [90]:
import collections
inlist = [
   ['bien', 'NP00000'],
   ['gracia', 'NCFP000'],
   ['estar', 'VAIP1S0'],
   ['bien', 'RG'],
   ['huevo', 'NCMS000'],
   ['calcio', 'NCMS000'],
   ['leche', 'NCFS000'],
   ['proteina', 'NCFS000'],
   ['francisco', 'NP00000'],
   ['ya', 'RG'],
   ['ser', 'VSIS3S0'],
   ['cosa', 'NCFS000']
]
#[(a,b,v) for (a,b),v in collections.Counter(c[0]  for c in inlist)]
collections.Counter(c[0]  for c in inlist)

Counter({'bien': 2,
         'gracia': 1,
         'estar': 1,
         'huevo': 1,
         'calcio': 1,
         'leche': 1,
         'proteina': 1,
         'francisco': 1,
         'ya': 1,
         'ser': 1,
         'cosa': 1})